# Inertial-Horizontal Velocity Sim

In [5]:
using Oceananigans
using Oceananigans.Units
using GLMakie

Nz = 100
Lz = 100
f = 1e-4

grid = RectilinearGrid(size=Nz, z=(-Lz, 0), topology=(Flat, Flat, Bounded))
coriolis = FPlane(; f)
model = HydrostaticFreeSurfaceModel(; grid, coriolis, tracers=(), buoyancy=nothing)

h = 20 # vertical scale of the inertial wave
uᵢ(z) = exp(-z^2 / 2h^2)
set!(model, u=uᵢ)

T = 2π / f # inertial period
Δt = 1e-2 * T
simulation = Simulation(model; Δt, stop_time=4T)

data = []

function save_data!(sim)
    u, v, w = sim.model.velocities
    ui = interior(u, 1, 1, :)
    vi = interior(v, 1, 1, :)
    push!(data, (t=time(sim), u=deepcopy(ui), v=deepcopy(vi)))
    return nothing
end

add_callback!(simulation, save_data!, TimeInterval(0.02T))

run!(simulation)

fig = Figure()
ax = Axis(fig[1, 1], xlabel="Velocities", ylabel="z (m)")

Nt = length(data)

n = Observable(1)
un = @lift data[$n].u
vn = @lift data[$n].v
tn = @lift data[$n].t

z = znodes(model.velocities.u)

lines!(ax, un, z, label="u")
lines!(ax, vn, z, label="v")
xlims!(ax, -1.1, 1.1)
axislegend(ax, position=:rb)

record(fig, "inertial_oscillation.mp4", 1:Nt, framerate=24) do nn
    n[] = nn
end

[ Info: Initializing simulation...
[ Info:     ... simulation initialization complete (27.322 ms)
[ Info: Executing initial time step...
[ Info:     ... initial time step complete (499.172 μs).
[ Info: Simulation is stopping after running for 676.780 ms.
[ Info: Simulation time 2.909 days equals or exceeds stop time 2.909 days.


"inertial_oscillation.mp4"

In [4]:
using PyCall
display = pyimport("IPython.display")
video_path = "/home/cl/Downloads/inertial_oscillation.mp4"
display.Video(video_path, embed=true, width=600)

PyObject <IPython.core.display.Video object>